# Import modules

In [2]:
!pip3 install pdflatex qiskit ipywidgets quantum-decomp pylatexenc import-ipynb
!wget https://raw.githubusercontent.com/hkhetawat/QArithmetic/master/QArithmetic.py
!wget https://raw.githubusercontent.com/hkhetawat/QArithmetic/master/qft.py

Defaulting to user installation because normal site-packages is not writeable


--2022-09-23 10:22:40--  https://raw.githubusercontent.com/hkhetawat/QArithmetic/master/QArithmetic.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13886 (14K) [text/plain]
Saving to: ‘QArithmetic.py.2’

QArithmetic.py.2    100%[===================>]  13,56K  --.-KB/s    in 0s      

2022-09-23 10:22:41 (36,4 MB/s) - ‘QArithmetic.py.2’ saved [13886/13886]

--2022-09-23 10:22:41--  https://raw.githubusercontent.com/hkhetawat/QArithmetic/master/qft.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1820 (1,8K) [text/plain]
Saving

In [1]:
import import_ipynb, qiskit, functools, QArithmetic
import numpy as np
import math
import matplotlib as plt
from itertools import chain
from qiskit import Aer, IBMQ, QuantumRegister, ClassicalRegister, QuantumCircuit, execute, assemble, transpile
from qiskit.providers.ibmq import least_busy
from qiskit.tools.monitor import job_monitor
from qiskit.providers.aer import noise
from qiskit.circuit import Gate
from qiskit.visualization import plot_histogram

In [2]:
class semigroupMembershipOracle():
    def __init__(self, soughtElement, *generators):
        
        self.soughtElement = soughtElement
        self.generators = generators # I'm asumming the input is like SMA.generators
    
    def substraction():
        '''operador de diferencia'''
        pass
    def addition():
        '''operador de suma'''
        pass
    def product():
        '''operador de producto'''
        pass
    def linearCombination():
        '''operador de combinación lineal'''
        pass
    

In [3]:
test1 = semigroupMembershipOracle(7, 3, 5)
test1.soughtElement
list(test1.generators)

[3, 5]

In [71]:
class groverPhase():
    
    '''Compactar en cajas la combinación lineal???'''
    
    def __init__(self, initH = False):
        self.initH = initH # Ensambla con el H inicial?
        
    def substraction():
        '''operador de diferencia'''
        pass
    def addition():
        '''operador de suma'''
        pass
    def product(self, qCircuit, threadsA, threadsB, threadsC):
        '''operador de producto'''
        n = len(threadsA)
        QArithmetic.mult(qCircuit, threadsA, threadsB, threadsC[:2*n], n)
        
        
    def linearCombination():
        '''operador de combinación lineal'''
        pass
    
    def semigroupMembershipOracle():
        pass # Ensambla el oráculo.
    
    def difussor():
        '''Crea un difusor en los hilos implicados'''
        pass
    
    def induceSuperposition(self, qCircuit, threads):
        
        if not self.initH:
            
            for i in range(len(threads)):
                qCircuit.h(threads[i])
            self.initH = True
            
    def assembleCircuit():
        '''Ensambla el circuito con una iter de oracle and difussor'''
        pass
        

In [72]:
class semigroupMembership(groverPhase):
    
    '''El numero opt. de iter (sol simple) es sqrt(2^n)'''
    
    def __init__(self, soughtElement, sizeOfLambda = 0, *generators):
        super().__init__()
        self.soughtElement = soughtElement
        self.generators = generators
        self.numberOfGenerators = len(generators)
        if sizeOfLambda==0:
            self.sizeOfLambda = max([math.ceil(math.log2(gen)) for gen in generators])
        else:
            self.sizeOfLambda = sizeOfLambda
            
        '''Numero de iteraciones del algoritmod e grover'''
        self.numbIter = math.floor(math.sqrt(self.numberOfGenerators*self.sizeOfLambda))
        
        
    def dec2binQR(self, qReg, number): # ****
        '''Toma un valor decimal y un registro, introduce el equiv. binairo en el registro.'''
        
        binExp = bin(number) #−> 0bxxxxx...
            
        if len(qReg)<len(binExp)-2:
            print(f"error al convertir en binario{len(qReg)} and {len(binExp)-2}")
            return
        i=0
        # De derecha a izquierda
        while binExp[-(i+1)]!='b':
            if binExp[-(i+1)]=='1':
              self.circ.x(qReg[i])
            i += 1
        del(i, binExp)
    
    def setUpThreads(self):
        
        sizeOfGenerators = max([math.ceil(math.log2(gen)) for gen in self.generators])
        sizeOfSought = math.ceil(math.log2(self.soughtElement))
        
        self.threadsOfGenerators = [QuantumRegister(sizeOfGenerators, f's{i}')
                                    for i in range(self.numberOfGenerators)]
        
        self.threadsOfLambda = [QuantumRegister(self.sizeOfLambda, f'lambda{i}')
                                    for i in range(self.numberOfGenerators)]
        
        self.threadsOfLinCom = [QuantumRegister(self.sizeOfLambda*2 + j, f'linearCom{j}')
                                for j in range(self.numberOfGenerators)]
        
        self.threadsOfSought = [QuantumRegister(sizeOfSought, 'a')]
        
        self.threadPhaseKickback = QuantumRegister(1, 'b')      
            
        self.circ = QuantumCircuit(*self.threadsOfGenerators, *self.threadsOfLambda,
                                   *self.threadsOfLinCom, *self.threadsOfSought, self.threadPhaseKickback,
                                   name='Semigroup Membership Algorithm')
        
        del(sizeOfGenerators, sizeOfSought)
        
    def setUpValues(self):
        
        for i in range(self.numberOfGenerators):
            self.dec2binQR(self.threadsOfGenerators[i], self.generators[i])
        self.dec2binQR(self.threadsOfSought[0], self.soughtElement)

    def setUpPhaseKickback(self):
        self.circ.x(self.threadPhaseKickback)
        self.circ.h(self.threadPhaseKickback)

In [73]:
test1 = semigroupMembership(6, 0, 5, 3) # The input of the generators isn't efficient. It's not flexible.
test1.setUpThreads() # Crea los hilos
test1.setUpValues() # Asigna valores a los generadores y el elemento buscado
test1.induceSuperposition(test1.circ, test1.threadsOfLambda)
test1.setUpPhaseKickback()
test1.product(test1.circ, test1.threadsOfGenerators[0], test1.threadsOfLambda[0], test1.threadsOfLinCom[0])

In [74]:
test1.threadsOfLambda[0]

QuantumRegister(3, 'lambda0')

In [75]:
test1.circ.draw('mpl')

In [187]:
del(test1)